In [ ]:
quiet_library <- function(...) {
    suppressPackageStartupMessages(library(...))
}
quiet_library(Seurat)
quiet_library(ggplot2)
quiet_library(Matrix)
quiet_library(H5weaver)
quiet_library(dplyr)
quiet_library(viridis)
quiet_library(harmony)
quiet_library(Nebulosa)

# Load Data

In [ ]:
sp_merge <- readRDS(file = '../../Seurat_Objects/sp_merge_gating_celltype_updated.rds')

In [ ]:
sp_merge <- SetIdent(sp_merge, value = 'gating_celltype')
cd4_mem <- subset(sp_merge, idents = c('CD4 CM','CD4 EM1','CD4 EM2','CD4 TEMRA','Treg'))

In [ ]:
cd4_mem$pure_celltype <- cd4_mem$gating_celltype

In [ ]:
cd4na <- readRDS(file = 'cd4_na_labeled.rds')

In [ ]:
cd4_merge <- merge(cd4na, cd4_mem)

In [ ]:
rm(sp_merge)
rm(cd4_mem)
rm(cd4na)

# Re-normalization

In [ ]:
DefaultAssay(cd4_merge) <- 'RNA'
cd4_merge <- NormalizeData(cd4_merge) %>% FindVariableFeatures() %>% ScaleData() %>% RunPCA()

In [ ]:
ElbowPlot(cd4_merge, ndims = 50)

In [ ]:
cd4_merge <- RunUMAP(cd4_merge, dims = 1:25)

In [ ]:
DimPlot(cd4_merge, group.by = 'pure_celltype')

# DEG Analysis

In [ ]:
cd4_merge <- SetIdent(cd4_merge, value = 'pure_celltype')
markers <- FindAllMarkers(cd4_merge)

In [ ]:
cd4_downsample <- subset(cd4_merge, downsample = 100)

In [ ]:
markers %>%
    group_by(cluster) %>%
    top_n(n = 10, wt = avg_log2FC) -> top10
DoHeatmap(cd4_downsample, features = top10$gene) + NoLegend()

# Average Expression of Markers

In [ ]:
degs_avg_exp <- AverageExpression(cd4_merge, assays = 'RNA', features = unique(markers$gene), group.by = 'pure_celltype')
degs_avg_exp <- as.data.frame(degs_avg_exp)

In [ ]:
head(degs_avg_exp)

In [ ]:
write.csv(markers, file = 'cd4_rna_degs.csv')

In [ ]:
markers <- read.csv(file='cd4_rna_degs.csv')

In [ ]:
length(unique(markers$gene))

# Pheatmap - Ext Fig 3A

In [ ]:
library(pheatmap)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 16)
cd4_pheatmap <- pheatmap(degs_avg_exp, cluster_rows = TRUE, cluster_cols = TRUE, scale = 'row', cellwidth = 50, cellheight = 0.5, show_rownames = FALSE, border_color = NA,
                         # annotation_row = cell_state_genes, annotation_colors = anno_colors,
                         color = colorRampPalette(c('#67a9cf','white','#d6604d'))(100))

In [ ]:
save_pheatmap_pdf <- function(x, filename, width=10, height=16) {
   stopifnot(!missing(x))
   stopifnot(!missing(filename))
   pdf(filename, width=width, height=height)
   grid::grid.newpage()
   grid::grid.draw(x$gtable)
   dev.off()
}
save_pheatmap_pdf(cd4_pheatmap, "plots/CD4_global_RNA_heatmap.pdf")